In [54]:
import numpy as np
from sklearn.datasets import make_friedman1
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, RMSprop, Adam, Optimizer
import plotly.graph_objs as go
from pathlib import Path

In [55]:
# I use my own GPU, I've set up CUDA on my machine
%set_env XLA_FLAGS=--xla_gpu_cuda_data_dir=/opt/cuda/

env: XLA_FLAGS=--xla_gpu_cuda_data_dir=/opt/cuda/


In [56]:

input_dim = 10
output_dim = 1
max_layer_scale_factor = 2

epochs = 100
batch_size = 1000

optimizers = [SGD, RMSprop, Adam]#[0:1]

learning_rates = [0.1, 0.01, 0.001]#[0:1]

architectures = [1, 5, 25]#[0:1]

Friedman dataset. 10 features. Regression target is given by 

$$y(X) = 10 * sin(pi * X[:, 0] * X[:, 1]) + 20 * (X[:, 2] - 0.5) ** 2 + 10 * X[:, 3] + 5 * X[:, 4] + noise * N(0, 1)$$

In [57]:
X, y = make_friedman1(
    n_samples=150000, n_features=input_dim, noise=1.0, random_state=42
)

In [58]:
X = MinMaxScaler().fit_transform(X)
y = MinMaxScaler().fit_transform(y.reshape(-1, 1)).ravel()

In [59]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=int(0.8 * len(X)), random_state=42
)

In [60]:
# I am not using sklearn LinearRegression. cause that's how I get history and learning curves
# And it works with the same plotting functions as DNN!
def compile_linear():
    model = Sequential()
    model.add(Dense(output_dim, activation="linear"))
    model.compile(
        optimizer="rmsprop",
        loss="mean_squared_error",
        metrics=["mape", "mse", "r2_score"],
    )
    return model

In [61]:
def compile(
    optimizer: Optimizer, learning_rate: float, hidden_layers: int
) -> Sequential:
    print(
        f"=== Compiling network with {hidden_layers} hidden layer(s), {optimizer.__name__} optimizer and LR={learning_rate} ==="
    )
    model = Sequential()

    # plus input and output layers
    total_layers = hidden_layers + 2
    for i in range(1, total_layers):

        # a little formula to make the network wider in the middle and narrower at the ends
        interpolation_index = np.interp(
            i, [0, total_layers - 1], [input_dim, output_dim]
        )
        layer_size = int(
            interpolation_index
            * (1.0 + np.sin(i / (total_layers - 1) * np.pi) * max_layer_scale_factor)
        )

        model.add(
            Dense(layer_size, activation="relu" if i != total_layers - 1 else "sigmoid")
        )

    optimizer_instance = optimizer(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer_instance,
        loss="mean_squared_error",
        metrics=["mape", "mse", "r2_score"],
    )

    return model

In [62]:
def train(model: Sequential):
    print(
        f"=== Training network with {len(model.layers) - 1} hidden layer(s), {model.optimizer.__class__.__name__} optimizer and LR={round(float(model.optimizer.learning_rate.numpy()), 3)} ==="
    )
    model.fit(
        X_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, y_test),
    )


In [63]:
def create_plot(model: Sequential):
    history = model.history.history

    r2_train = history["r2_score"]
    r2_val = history["val_r2_score"]
    mape_train = history["mape"]
    mape_val = history["val_mape"]
    mse_train = history["mse"]
    mse_val = history["val_mse"]

    train_step = np.arange(1, epochs + 1)

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=train_step, y=r2_train, mode="lines", name="Training R2", yaxis="y1"
        )
    )
    fig.add_trace(
        go.Scatter(
            x=train_step, y=r2_val, mode="lines", name="Validation R2", yaxis="y1"
        )
    )

    fig.add_trace(
        go.Scatter(
            x=train_step, y=mape_train, mode="lines", name="Training MAPE", yaxis="y2"
        )
    )
    fig.add_trace(
        go.Scatter(
            x=train_step, y=mape_val, mode="lines", name="Validation MAPE", yaxis="y2"
        )
    )

    fig.add_trace(
        go.Scatter(
            x=train_step, y=mse_train, mode="lines", name="Training MSE", yaxis="y3"
        )
    )
    fig.add_trace(
        go.Scatter(
            x=train_step, y=mse_val, mode="lines", name="Validation MSE", yaxis="y3"
        )
    )

    # Different y-axes for different metrics
    fig.update_layout(
        yaxis=dict(
            title="R2",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4"),
        ),
        yaxis2=dict(
            title="MAPE",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            anchor="free",
            overlaying="y",
            autoshift=True,
        ),
        yaxis3=dict(
            title="MSE",
            titlefont=dict(color="#d62728"),
            tickfont=dict(color="#d62728"),
            anchor="free",
            overlaying="y",
            autoshift=True,
        ),
    )

    fig.update_layout(
        title=f"DNN with {len(model.layers) - 1} hidden Layer(s), {model.optimizer.__class__.__name__} optimizer, learning rate = {round(float(model.optimizer.learning_rate.numpy()), 3)}",
        xaxis_title="Epoch",
        legend_title="Metric",
    )

    return fig

In [64]:
linreg = compile_linear()
train(linreg)
fig = create_plot(linreg)

fig.show()

=== Training network with 0 hidden layer(s), RMSprop optimizer and LR=0.001 ===
Epoch 1/100


120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1082 - mape: 1092.8607 - mse: 0.1082 - r2_score: -3.4686 - val_loss: 0.0724 - val_mape: 48.2630 - val_mse: 0.0724 - val_r2_score: -1.9974
Epoch 2/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0633 - mape: 5355.1587 - mse: 0.0633 - r2_score: -1.6021 - val_loss: 0.0405 - val_mape: 36.2074 - val_mse: 0.0405 - val_r2_score: -0.6783
Epoch 3/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0349 - mape: 5799.1558 - mse: 0.0349 - r2_score: -0.4481 - val_loss: 0.0209 - val_mape: 26.1589 - val_mse: 0.0209 - val_r2_score: 0.1337
Epoch 4/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 959us/step - loss: 0.0178 - mape: 2318.9890 - mse: 0.0178 - r2_score: 0.2645 - val_loss: 0.0109 - val_mape: 18.8899 - val_mse: 0.0109 - val_r2_score: 0.5474
Epoch 5/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0096 - mape: 244.4223 - mse: 0.0096 - r2_score: 0.6034 - val_loss: 0.0074 - val_mape: 15.1681 - val_mse: 0.0074 - val_r2_score: 0.6916
Epoch 6/100

In [65]:
# we'll store our models and plots here
Path("trained").mkdir(parents=True, exist_ok=True)

In [66]:
for optimizer in optimizers:
    for lr in learning_rates:
        for hidden_layers in architectures:
            model_name = f"trained/model_{optimizer.__name__}_{lr}_{hidden_layers}.keras"

            model = compile(optimizer, lr, hidden_layers)
            train(model)
            
            model.save(model_name)
            
            fig = create_plot(model)
            
            fig.write_html(
                f"trained/model_{model.optimizer.__class__.__name__}_{round(float(model.optimizer.learning_rate.numpy()), 3)}_{len(model.layers) - 1}.html"
            )
            fig.show()  

=== Compiling network with 1 hidden layer(s), SGD optimizer and LR=0.1 ===
=== Training network with 1 hidden layer(s), SGD optimizer and LR=0.1 ===
Epoch 1/100


120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0366 - mape: 3367.1392 - mse: 0.0366 - r2_score: -0.5198 - val_loss: 0.0250 - val_mape: 32.6059 - val_mse: 0.0250 - val_r2_score: -0.0374
Epoch 2/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - loss: 0.0237 - mape: 5322.2480 - mse: 0.0237 - r2_score: 0.0173 - val_loss: 0.0208 - val_mape: 29.9120 - val_mse: 0.0208 - val_r2_score: 0.1399
Epoch 3/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 948us/step - loss: 0.0201 - mape: 5968.2734 - mse: 0.0201 - r2_score: 0.1715 - val_loss: 0.0182 - val_mape: 27.9362 - val_mse: 0.0182 - val_r2_score: 0.2470
Epoch 4/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 911us/step - loss: 0.0176 - mape: 10005.1680 - mse: 0.0176 - r2_score: 0.2678 - val_loss: 0.0162 - val_mape: 26.3478 - val_mse: 0.0162 - val_r2_score: 0.3278
Epoch 5/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 955us/step - loss: 0.0159 - mape: 3652.7581 - mse: 0.0159 - r2_score: 0.3428 - val_loss: 0.0146 - val_mape: 24.9018 - val_mse: 0.0146 - val_r2_score: 0.3945
Epoch 

=== Compiling network with 5 hidden layer(s), SGD optimizer and LR=0.1 ===
=== Training network with 5 hidden layer(s), SGD optimizer and LR=0.1 ===
Epoch 1/100


I0000 00:00:1710713968.491258  504798 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_248', 8 bytes spill stores, 8 bytes spill loads

I0000 00:00:1710713968.735391  504801 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_287', 876 bytes spill stores, 752 bytes spill loads

I0000 00:00:1710713968.910267  504793 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_114', 52 bytes spill stores, 52 bytes spill loads

I0000 00:00:1710713969.044678  504795 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_261', 456 bytes spill stores, 456 bytes spill loads

I0000 00:00:1710713969.307637  504791 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_248', 604 bytes spill stores, 604 bytes spill loads

I0000 00:00:1710713969.374609  

120/120 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0248 - mape: 4049.7871 - mse: 0.0248 - r2_score: -0.0234 - val_loss: 0.0241 - val_mape: 31.8517 - val_mse: 0.0241 - val_r2_score: 0.0018
Epoch 2/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0240 - mape: 2069.5017 - mse: 0.0240 - r2_score: 0.0078 - val_loss: 0.0234 - val_mape: 31.6799 - val_mse: 0.0234 - val_r2_score: 0.0305
Epoch 3/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0233 - mape: 1281.3517 - mse: 0.0233 - r2_score: 0.0386 - val_loss: 0.0226 - val_mape: 31.0672 - val_mse: 0.0226 - val_r2_score: 0.0644
Epoch 4/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0225 - mape: 3913.1604 - mse: 0.0225 - r2_score: 0.0730 - val_loss: 0.0216 - val_mape: 30.3200 - val_mse: 0.0216 - val_r2_score: 0.1068
Epoch 5/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 990us/step - loss: 0.0212 - mape: 13203.2607 - mse: 0.0212 - r2_score: 0.1203 - val_loss: 0.0199 - val_mape: 29.0437 - val_mse: 0.0199 - val_r2_score: 0.1771
Epoch 6/100
1

...

=== Compiling network with 5 hidden layer(s), Adam optimizer and LR=0.01 ===
=== Training network with 5 hidden layer(s), Adam optimizer and LR=0.01 ===
Epoch 1/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0098 - mape: 5419.0645 - mse: 0.0098 - r2_score: 0.5945 - val_loss: 0.0020 - val_mape: 8.0947 - val_mse: 0.0020 - val_r2_score: 0.9176
Epoch 2/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0018 - mape: 1477.0881 - mse: 0.0018 - r2_score: 0.9257 - val_loss: 0.0018 - val_mape: 8.0939 - val_mse: 0.0018 - val_r2_score: 0.9254
Epoch 3/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0014 - mape: 1972.0770 - mse: 0.0014 - r2_score: 0.9404 - val_loss: 0.0013 - val_mape: 6.7115 - val_mse: 0.0013 - val_r2_score: 0.9448
Epoch 4/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0015 - mape: 1514.7765 - mse: 0.0015 - r2_score: 0.9387 - val_loss: 0.0013 - val_mape: 6.7208 - val_mse: 0.0013 - val_r2_score: 0.9460
Epoch 5/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - 

=== Compiling network with 25 hidden layer(s), Adam optimizer and LR=0.01 ===
=== Training network with 25 hidden layer(s), Adam optimizer and LR=0.01 ===
Epoch 1/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.0242 - mape: 1224.3534 - mse: 0.0242 - r2_score: -2.7013e-04 - val_loss: 0.0241 - val_mape: 31.6917 - val_mse: 0.0241 - val_r2_score: -4.7565e-05
Epoch 2/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0242 - mape: 352.5218 - mse: 0.0242 - r2_score: -3.7521e-04 - val_loss: 0.0241 - val_mape: 31.8253 - val_mse: 0.0241 - val_r2_score: -4.1246e-05
Epoch 3/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0241 - mape: 6107.6328 - mse: 0.0241 - r2_score: -7.4831e-05 - val_loss: 0.0241 - val_mape: 31.7351 - val_mse: 0.0241 - val_r2_score: -4.5300e-06
Epoch 4/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0241 - mape: 4433.6597 - mse: 0.0241 - r2_score: -1.4065e-04 - val_loss: 0.0241 - val_mape: 31.5719 - val_mse: 0.0241 - val_r2_score: -3.8373e-04
Epoch 5/

=== Compiling network with 1 hidden layer(s), Adam optimizer and LR=0.001 ===
=== Training network with 1 hidden layer(s), Adam optimizer and LR=0.001 ===
Epoch 1/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0252 - mape: 2700.5193 - mse: 0.0252 - r2_score: -0.0379 - val_loss: 0.0115 - val_mape: 21.4544 - val_mse: 0.0115 - val_r2_score: 0.5231
Epoch 2/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0105 - mape: 8364.8184 - mse: 0.0105 - r2_score: 0.5666 - val_loss: 0.0084 - val_mape: 17.4833 - val_mse: 0.0084 - val_r2_score: 0.6507
Epoch 3/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0081 - mape: 1298.0178 - mse: 0.0081 - r2_score: 0.6656 - val_loss: 0.0071 - val_mape: 15.7252 - val_mse: 0.0071 - val_r2_score: 0.7041
Epoch 4/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0070 - mape: 3488.7251 - mse: 0.0070 - r2_score: 0.7090 - val_loss: 0.0066 - val_mape: 14.6368 - val_mse: 0.0066 - val_r2_score: 0.7264
Epoch 5/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/

=== Compiling network with 5 hidden layer(s), Adam optimizer and LR=0.001 ===
=== Training network with 5 hidden layer(s), Adam optimizer and LR=0.001 ===
Epoch 1/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0203 - mape: 3248.9729 - mse: 0.0203 - r2_score: 0.1564 - val_loss: 0.0124 - val_mape: 24.3282 - val_mse: 0.0124 - val_r2_score: 0.4849
Epoch 2/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0115 - mape: 7186.2402 - mse: 0.0115 - r2_score: 0.5281 - val_loss: 0.0087 - val_mape: 20.3431 - val_mse: 0.0087 - val_r2_score: 0.6376
Epoch 3/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0079 - mape: 2832.7100 - mse: 0.0079 - r2_score: 0.6758 - val_loss: 0.0056 - val_mape: 16.2362 - val_mse: 0.0056 - val_r2_score: 0.7675
Epoch 4/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0052 - mape: 236.0970 - mse: 0.0052 - r2_score: 0.7839 - val_loss: 0.0041 - val_mape: 13.8373 - val_mse: 0.0041 - val_r2_score: 0.8283
Epoch 5/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/st

=== Compiling network with 25 hidden layer(s), Adam optimizer and LR=0.001 ===
=== Training network with 25 hidden layer(s), Adam optimizer and LR=0.001 ===
Epoch 1/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - loss: 0.0241 - mape: 4128.6562 - mse: 0.0241 - r2_score: -2.2896e-04 - val_loss: 0.0241 - val_mape: 31.7550 - val_mse: 0.0241 - val_r2_score: 1.7881e-06
Epoch 2/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0243 - mape: 16216.3516 - mse: 0.0243 - r2_score: -4.6796e-05 - val_loss: 0.0241 - val_mape: 31.8037 - val_mse: 0.0241 - val_r2_score: -1.7524e-05
Epoch 3/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0239 - mape: 5178.5630 - mse: 0.0239 - r2_score: -6.9951e-05 - val_loss: 0.0241 - val_mape: 31.7271 - val_mse: 0.0241 - val_r2_score: -9.6560e-06
Epoch 4/100
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0242 - mape: 783.9232 - mse: 0.0242 - r2_score: -5.0123e-05 - val_loss: 0.0241 - val_mape: 31.8031 - val_mse: 0.0241 - val_r2_score: -1.6928e-05
Epoch 

### We can thus conclude that:
- DNN with 1 and 5 hidden layers performed better than baseline LR, confirming non-linearity of the dataset (as obvious from the cosine therm in the Friedman formula).
- In the performed results, the best number of hidden layers is 5
- Networks with 25 layers perform the worst, never reaching lower than 0.2 MSE. R2 scores for the configuration suggest it is worse fit than horizontal line.
- Picking higher learning rate sometimes results in inability to find the minimum, thus remaining stuck at a high loss, about 0.2 MSE, like with 5 hidden layers, Adam optimizer and LR=0.1
- The absolute best pick of hyperparameters is 5 hidden layers, Adam optimizer and LR=0.001. With it, loss is 0.001, R2 score is 0.95+ 
- Number of layers higher than 5 causes training process to become unstable. It does not find the minimum, learning process is erratic.
- Data distribution influenced validity of some scores (unnnormalized data yielded NaN values) and loss descent rates. Normalizing both input and output data relative to each dimension yielded the best results